In [ ]:
import random
from math import pi

%run Components.ipynb
%run Setting.ipynb

In [ ]:
class Item:
    component_list = []
    allowed_components = []
    creation_function = None
    system = None
    num_components = None
    ccds = None
    inputs = []
    
    # inputs: array of {"data": matrix, "position": tuple}
    # ccds: array of CcdComponent
    def __init__(self, allowed_components, inputs, num_components, ccds):
        self.creation_function = self.random_component_ray_direction_position
        self.allowed_components = allowed_components
        self.num_components = num_components
        self.inputs = inputs
        self.ccds = ccds
    
    
    def get_system_complist(self):
        complist = []
        last_loc = None
        last_loc2 = None
        last_rotation = None
        for i in self.component_list:
            complist.append((i.get_instance(), i.get_loc(), i.get_rotation()))
        
        if len(self.component_list) > 1:
            last_loc = self.component_list[-1].get_loc()
            last_loc2 = self.component_list[-2].get_loc()
            last_rotation = self.component_list[-1].get_rotation()
            diff = (x - y for x,y in zip(last_loc, last_loc2))
            new_loc = (x + y for (x,y) in zip(last_loc, diff))
        return complist

    
    def convert_input_to_ray(self, input_matrix, position):
        w, h = len(input_matrix[0]), len(input_matrix)
        ray = parallel_beam_c(num_rays=(w, h), origin=position)
        for index, r in enumerate(ray):
            x = index // w
            y = index % w
            if input_matrix[x][y] == 0:
                r.intensity = 0.5
        return ray

        
    def evaluate(self):
        self.creation_function(self.num_components)
        return self.get_final_ray()
    
    
    def get_output(self):
        outputs = []
        for ccd in self.ccds:
            outputs.append(ccd.get_instance().hit_list)
        return outputs
    
    
    def get_final_ray(self):
        if self.system:
            self.system.reset()
            self.system.clear()
            self.system.clear_ray_list()
        rays = []
        for inp in inputs:
            rays.append(self.convert_input_to_ray(inp["data"], inp["position"]))
        
        complist = self.get_system_complist()
        for ccd in self.ccds:
            complist.append((ccd.get_instance(), ccd.get_loc(), ccd.get_rotation()))
        
        system = System(complist=complist, n=1)
        for i in rays:
            system.ray_add(i)
        system.propagate()
        self.system = system
        final_rays = []
        for i in rays:
            for j in i:
                try:
                    final_rays += j.get_final_rays()
                except Exception as e:
                    print(e)
                    return None
        return final_rays
       
    
    def random_component_random_position(self, num_components):
        self.component_list = []
        rotation_options = ROTATION_OPTIONS
        for i in range(num_components):
            component_class = random.choice(self.allowed_components)
            positions = (random.randint(0, LOC_LIMIT[0]), random.randint(0, LOC_LIMIT[1]), random.randint(0, LOC_LIMIT[2]))
            rotations = tuple(random.choices(rotation_options, k=3))
            component_object = component_class(positions, rotations)
            self.component_list.append(component_object)
    
    
    def random_component_ray_direction_position(self, num_components):
        self.component_list = []
        rotation_options = ROTATION_OPTIONS
        for i in range(num_components):
            component_class = random.choice(self.allowed_components)
            final_rays = self.get_final_ray()
            if not final_rays:
                i -= 1
                continue
            final_ray = final_rays[len(final_rays) // 2]
            coeff = random.randint(10, 20)
            positions = [a + coeff * b for a,b in zip(final_ray.pos, final_ray.dir)]
            rotations = (random.choices(rotation_options, k=1)[0], 0, 0)
            component_object = component_class(positions, rotations)
            self.component_list.append(component_object)
    
    
    def mutate(self):
        should_mutate = random.random() > 0.8
        if not should_mutate:
            return
        index_to_mutate = random.randint(0, len(self.component_list) - 1)
        cmp = self.component_list[index_to_mutate]
        cmp.mutate()
    
    
    def crossover(self, another_item):
        rand_number = random.random()
        if rand_number > 0.5:
            return self
        else:
            return another_item

In [ ]:
# Test code to create a priscope with genetic algorithm

allowed_components = [
    MirrorComponent
]

items = []
inputs = [
    {"data": [[1, 1], [1, 1], [1, 1]], "position": (0,0,0)},
]

def generate_population(num=100):
    global items
    for i in range(num):
        ccds = [CcdComponent(loc=(0, 10, 10), rotation=(0,0,0))]
        item = Item(allowed_components, inputs, 2, ccds)
        item.evaluate()
        final_rays = item.get_output()
        count = 0
        if final_rays is not None:
            count = len(final_rays)
            items.append((abs(count - 4), item))
            if i % 50 == 0:
                print(f"{i}. Item created")


def sort_population():
    global items
    items = sorted(items, key=lambda x: x[0], reverse=True)
    items = [x[1] for x in items]


def crossover():
    global items
    seventy_percent = round(0.7 * len(items))
    twenty_percent = round(0.2 * len(items))
    items = items[:seventy_percent]
    for i in range(1, twenty_percent, 2):
        items[i] = items[i].crossover(items[i-1])


def mutate():
    global items
    for item in items:
        item.mutate()


def evaluate():
    global items
    new_items = []
    for item in items:
        item.evaluate()
        final_rays = item.get_output()
        count = 0
        if final_rays is not None:
            count = len(final_rays)
        new_items.append((abs(count - 4), item))
    items = new_items
        
    

generate_population()
print("Population Generated!")

for i in range(3):
    sort_population()
    crossover()
    mutate()
    evaluate()
    print(f"{i}th generation...")

sort_population()
final_rays = items[0].get_output()
print(len(final_rays))
Plot3D(items[0].system,center=(0,25,50), size=(150,100),scale=4,rot=[(0,-pi/2,0)])